In [ ]:
import pathlib
import requests

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import compose, datasets, ensemble, linear_model, metrics
from sklearn import model_selection, neighbors, pipeline
from sklearn import preprocessing, svm, tree

# Ensemble Learning

Building a model on top of many other models is called [ensemble](https://scikit-learn.org/stable/modules/ensemble.html) learning and it is often a great approach to improve the predictions of your machine learning pipeline.

## Load and prepare the data

In [ ]:
diabetes_data = datasets.load_diabetes(
    as_frame=True,
    scaled=False
)

In [ ]:
features_df = diabetes_data.data
target = diabetes_data.target

In [ ]:
features_df.head()

In [ ]:
target

### Data preprocessing

In [ ]:
transformer_0 = compose.make_column_transformer(
    (
        preprocessing.OneHotEncoder(
            drop="first",
            dtype=np.uint8,
            sparse_output=False,
        ),
        ["sex"]
    ),
    remainder="drop",
    verbose=True,
    verbose_feature_names_out=False
)

transformer_1 = compose.make_column_transformer(
    (
        preprocessing.StandardScaler(),
        ["age", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"]
    ),
    remainder="drop",
    verbose=True,
    verbose_feature_names_out=False
)

features_preprocessor = pipeline.make_union(
    transformer_0,
    transformer_1,
    verbose=True,
    n_jobs=-1
).set_output(transform="pandas")

target_preprocessor = preprocessing.FunctionTransformer(
    func=np.log,
    inverse_func=np.exp
)

### Feature engineering

In [ ]:
feature_engineering = preprocessing.PolynomialFeatures(
    degree=2,
    include_bias=False,
    interaction_only=False
).set_output(transform="pandas")


## Voting

In [ ]:
ensemble.VotingRegressor?

In [ ]:
voting_regressor = ensemble.VotingRegressor(
    estimators=[
        ("sgd_regressor", linear_model.SGDRegressor()),
        ("k_neighbors_regressor", neighbors.KNeighborsRegressor()),
        ("linear_svr", svm.LinearSVR()),
        ("tree", tree.DecisionTreeRegressor()),
    ],
    weights=None,
    n_jobs=-1,
    verbose=True
)

voting_regressor_pipeline = pipeline.make_pipeline(
    features_preprocessor,
    feature_engineering,
    voting_regressor,
    verbose=True,
)

In [ ]:
voting_regressor_pipeline

In [ ]:
ensemble_scores = model_selection.cross_val_score(
    voting_regressor_pipeline,
    features_df,
    target,
    cv=5,
    n_jobs=1,
    scoring="neg_mean_squared_error"
)

In [ ]:
np.sqrt(-ensemble_scores)

### Excercise

Use cross validation to score each of the individual estimators included in the voting regressor above. Compare the results of our ensemble with the best individual model.

#### Solution

In [ ]:
_sgd_regressor_pipeline = pipeline.make_pipeline(
    features_preprocessor,
    feature_engineering,
    linear_model.SGDRegressor()
)

sgd_regressor_scores = model_selection.cross_val_score(
    _sgd_regressor_pipeline,
    features_df,
    target,
    cv=5,
    n_jobs=1,
    scoring="neg_mean_squared_error"
)

print(np.sqrt(-sgd_regressor_scores))

In [ ]:
_sgd_regressor_pipeline = pipeline.make_pipeline(
    features_preprocessor,
    feature_engineering,
    linear_model.SGDRegressor()
)

sgd_regressor_scores = model_selection.cross_val_score(
    _sgd_regressor_pipeline,
    features_df,
    target,
    cv=5,
    n_jobs=1,
    scoring="neg_mean_squared_error"
)

print(np.sqrt(-sgd_regressor_scores))

In [ ]:
_linear_svr_pipeline = pipeline.make_pipeline(
    features_preprocessor,
    feature_engineering,
    svm.LinearSVR()
)

linear_svr_scores = model_selection.cross_val_score(
    _linear_svr_pipeline,
    features_df,
    target,
    cv=5,
    n_jobs=1,
    scoring="neg_mean_squared_error"
)

print(np.sqrt(-linear_svr_scores))

In [ ]:
_neighbors_pipeline = pipeline.make_pipeline(
    features_preprocessor,
    feature_engineering,
    neighbors.KNeighborsRegressor()
)

neighbors_scores = model_selection.cross_val_score(
    _neighbors_pipeline,
    features_df,
    target,
    cv=5,
    n_jobs=1,
    scoring="neg_mean_squared_error"
)

print(np.sqrt(-neighbors_scores))

In [ ]:
_tree_pipeline = pipeline.make_pipeline(
    features_preprocessor,
    feature_engineering,
    tree.DecisionTreeRegressor()
)

tree_scores = model_selection.cross_val_score(
    _tree_pipeline,
    features_df,
    target,
    cv=5,
    n_jobs=1,
    scoring="neg_mean_squared_error"
)

print(np.sqrt(-tree_scores))

## Bagging and Pasting

In [ ]:
ensemble.BaggingRegressor?

In [ ]:
bagging_regressor = ensemble.BaggingRegressor(
    estimator=tree.DecisionTreeRegressor(),
    n_estimators=10,
    max_samples=0.8,
    bootstrap=True,
    max_features=1.0,
    bootstrap_features=False,
    oob_score=True,
    n_jobs=-1,
    random_state=42,
    verbose=True
)

In [ ]:
bagging_regressor

In [ ]:
_ = bagging_regressor.fit(features_df, target)

In [ ]:
oob_predictions = bagging_regressor.oob_prediction_
metrics.mean_squared_error(
    target,
    oob_predictions,
    squared=False
)

## Random Forests

Let’s try the [`ensemble.RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). Random forests work by training many decision trees on random subsets of the features, then averaging the predictions made by each of the decision trees to arrive at an overall prediction.

In [ ]:
_random_forest_pipeline = pipeline.make_pipeline(
    features_preprocessor,
    feature_engineering,
    ensemble.RandomForestRegressor()
)

random_forest_scores = model_selection.cross_val_score(
    _random_forest_pipeline,
    features_df,
    target,
    cv=5,
    n_jobs=1,
    scoring="neg_mean_squared_error"
)

print(np.sqrt(-random_forest_scores))

### Exercise

Compare the performance of the ExtraTreesRegressor with the RandomForestRegressor fit above.

#### Solution

In [ ]:
_extra_trees_pipeline = pipeline.make_pipeline(
    features_preprocessor,
    feature_engineering,
    ensemble.ExtraTreesRegressor()
)

extra_trees_scores = model_selection.cross_val_score(
    _extra_trees_pipeline,
    features_df,
    target,
    cv=5,
    n_jobs=1,
    scoring="neg_mean_squared_error"
)

print(np.sqrt(-extra_trees_scores))

### Exercise

Tune the hyperparameters of either the RandomForestRegressor or the ExtraTreesRegressor and see if you can get even better performance.

#### Solution